In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sys
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm
from catboost import CatBoostRegressor
import json
import ast
import re
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import f_classif, mutual_info_classif
from collections import Counter
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import BaggingRegressor

In [ ]:
def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))

In [ ]:
print('Python       :', sys.version.split('\n')[0])
print('Numpy        :', np.__version__)

In [ ]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

In [ ]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

# Setup

In [ ]:
VERSION    = 16
# DIR_TRAIN  = '../input/project6-train/' # подключил к ноутбуку внешний датасет
DIR_TRAIN  = '../input/project61/' # 
DIR_TEST   = '../input/sf-dst-car-price-prediction/'
VAL_SIZE   = 0.33   # 33%
N_FOLDS    = 5

# CATBOOST
ITERATIONS = 2000
LR         = 0.1

# Data

In [ ]:
!ls '../input'

**Парсинг  url**

In [ ]:
# cars_urls = {}
# url = 'https://auto.ru/cars/'
# for car_brand in brands:
#     brand_url = url + car_brand + '/used/?page='
#     cars_urls[car_brand] = []
#     print(car_brand)
#     for number in range(1, 100):
#         response = requests.get(brand_url+str(number), headers={'User-Agent': 'Mozilla/5.0'})
#         page = BeautifulSoup(response.text, 'html.parser')
#         link_list = page.find_all('a', class_='Link ListingItemTitle-module__link')
#         if len(link_list) !=0:
#             for link in link_list:
#                 cars_urls[car_brand].append(link['href'])
#         else:
#             break
# with open('data.json', 'w') as fp:
#     json.dump(cars_urls, fp)

Парсинг данных

In [ ]:
# counter = 0

# final_dict['car'] = []
# for url in cars_urls['url']:
#     url_dict = {}
#     response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
#     response.encoding = 'utf-8'
#     page = BeautifulSoup(response.text, 'html.parser')
#     if 'Этот автомобиль уже продан' in page.text:
#         continue
#     else:
#         try:
#             first_dict = ast.literal_eval(str(page.find_all('script', attrs = {'type':'application/ld+json'})[0])[35:-9])
#         except IndexError:
#             continue
#         url2 = ast.literal_eval(str(page.find_all('script', attrs = {'type':'application/ld+json'})[0])[35:-9])['offers']['url']
#         if url != url2:
#             continue
#         else:
#             try:
#                 url_dict['bodyType'] = first_dict['bodyType']
#                 url_dict['brand'] = first_dict['brand']
#                 url_dict['car_url'] = url
#                 url_dict['color'] = first_dict['color']
#                 url_dict['description'] = first_dict['description']
#                 url_dict['engineDisplacement'] = first_dict['vehicleEngine']['engineDisplacement']
#                 url_dict['enginePower'] = first_dict['vehicleEngine']['enginePower']
#                 url_dict['fuelType'] = first_dict['vehicleEngine']['fuelType']
#                 url_dict['mileage'] = int(page.find('li', class_ = 'CardInfoRow CardInfoRow_kmAge').find_all('span', class_='CardInfoRow__cell')[1].text.replace(u'\xa0', u' ')[:-3].replace(' ', ''))
#                 url_dict['modelDate'] = first_dict['modelDate']
#                 url_dict['model_name'] = str(page.find('script', attrs = {'type':'application/json', 'id':'initial-state'})).partition('"model_info":{"code":"')[2].partition('","name":"')[0]
#                 url_dict['name'] = str(page.find('script', attrs = {'type':'application/json', 'id':'initial-state'})).partition('"tech_param":')[2].partition('"human_name":"')[2].partition(')')[0]+')'
#                 url_dict['numberOfDoors'] = first_dict['numberOfDoors']
#                 url_dict['productionDate'] = first_dict['productionDate']
#                 url_dict['numberOfDoors'] = first_dict['numberOfDoors']
#                 url_dict['vehicleConfiguration'] = first_dict['vehicleConfiguration']
#                 url_dict['vehicleTransmission'] = first_dict['vehicleTransmission']
#                 url_dict['Владельцы'] = page.find('li', class_ = 'CardInfoRow CardInfoRow_ownersCount').find_all('span', class_='CardInfoRow__cell')[1].text.replace(u'\xa0', u' ')
#                 url_dict['ПТС'] = page.find('li', class_ = 'CardInfoRow CardInfoRow_pts').find_all('span', class_='CardInfoRow__cell')[1].text
#                 url_dict['Привод'] = page.find('li', class_ = 'CardInfoRow CardInfoRow_drive').find_all('span', class_='CardInfoRow__cell')[1].text
#                 url_dict['Руль'] = page.find('li', class_ = 'CardInfoRow CardInfoRow_wheel').find_all('span', class_='CardInfoRow__cell')[1].text
#                 complectation = []
#                 for i in page.find_all('li', class_='ComplectationGroups__itemContentEl'):
#                     complectation.append(i.text)
#                 url_dict['complectation'] = complectation
#                 super_gen = page.find('a', class_='Link SpoilerLink CardCatalogLink SpoilerLink_type_default')['href']
#                 response_1 = requests.get(super_gen, headers={'User-Agent': 'Mozilla/5.0'})
#                 response_1.encoding = 'utf-8'
#                 page_1 = BeautifulSoup(response_1.text, 'html.parser')
#                 for i in page_1.find_all('dd', class_='list-values__value'):
#                     if ' с' in i.text:
#                         url_dict['acceleration'] = i.text[:-2]
#                 url_dict.setdefault('acceleration', np.nan)
#                 ls = []
#                 for i in page_1.find_all('dd', class_='list-values__value'):
#                     if ' л' in i.text:
#                         ls.append(i.text)
#                     if len(ls) == 3:
#                         url_dict['fuel_rate'] = ls[2][:-2]
#                 url_dict.setdefault('fuel_rate', np.nan)
#                 url_dict['Price'] = int(page.find('span', class_ = 'OfferPriceCaption__price').text.replace(u'\xa0', u' ')[:-2].replace(' ', ''))
#                 final_dict['car'].append(url_dict)
#                 counter+=1
#                 if counter%100 == 0:
#                     print(counter)
#             except (KeyError, ValueError, AttributeError) as e:
#                 continue
# print('end')

In [ ]:

train = pd.read_csv(DIR_TRAIN+'project6_train_6.csv', low_memory=False) # датасет для обучения модели
test = pd.read_csv(DIR_TEST+'test.csv')
sample_submission = pd.read_csv(DIR_TEST+'sample_submission.csv')

In [ ]:
train.shape

In [ ]:
# убираем лишние колонки
train.drop(['complectation', 'acceleration', 'fuel_rate', 'car_url'], axis=1, inplace=True)
test.drop(['car_url', 'complectation_dict', 'equipment_dict', 'image', 'parsing_unixtime', 
           'priceCurrency', 'sell_id', 'super_gen', 'vendor', 'Владение', 'Состояние', 'Таможня', 'model_info'], axis=1, inplace=True)

In [ ]:
# заполняем пропуски
test['ПТС'].fillna('Оригинал', inplace=True)

In [ ]:
# Стандартизируем данные на тесте и в трейне
train["Владельцы"] = train["Владельцы"].apply(lambda x: '1\xa0владелец' if x == '1 владелец' else '2\xa0владельца' if x == '2 владельца' else x)
train["Владельцы"] = train["Владельцы"].apply(lambda x: 1 if x == '1\xa0владелец' else 2 if x == '2\xa0владельца' else 3)
test["Владельцы"] = test["Владельцы"].apply(lambda x: 1 if x == '1\xa0владелец' else 2 if x == '2\xa0владельца' else 3)

In [ ]:
# Лучшая корреляция с price если брать mileage в сетпени 0.5 и логарфим price
train['mileage'] = train['mileage']**(0.5)
test['mileage'] = test['mileage']**(0.5)
train['Price'] = np.log(train['Price'])

Новые признаки из описания машин


In [ ]:
def new(x):
    new =['новый', 'новая', 'новые']
    for i in new:
        if i in x:
            return 1
        else:
            continue
    return 0

In [ ]:
def condition_1(x):
    if 'идеал' in x:
        return 4
    elif 'отличн' in x:
        return 3
    elif 'хорош' in x:
        return 2
    else:
        return 1

In [ ]:
train.description = train.description.apply(lambda x: x.lower())
test.description = test.description.apply(lambda x: x.lower())

In [ ]:
train['новый'] = train.description.apply(lambda x: new(x))
test['новый'] = test.description.apply(lambda x: new(x))

In [ ]:
train['состояние'] = train.description.apply(lambda x: condition_1(x))
test['состояние'] = test.description.apply(lambda x: condition_1(x))

In [ ]:
train['inuse']= train.productionDate.apply(lambda x: (2021-x)**0.5)
test['inuse']= test.productionDate.apply(lambda x: (2021-x)**0.5)

In [ ]:
train['гараж'] = train.description.apply(lambda x: 1 if 'гараж' in x else 0)
train['подарок'] = train.description.apply(lambda x: 1 if 'подарок' in x else 0)
train['торг'] = train.description.apply(lambda x: 1 if 'торг' in x else 0)
train['шины'] = train.description.apply(lambda x: 1 if ('шин' in x) or ('резин' in x) else 0)
train['дилер'] = train.description.apply(lambda x: 1 if 'дилер' in x else 0)
train['подогрев'] = train.description.apply(lambda x: 1 if 'подогрев' in x else 0)
train['обмен'] = train.description.apply(lambda x: 1 if 'обмен' in x else 0)
train['обслуж'] = train.description.apply(lambda x: 1 if 'обслуж' in x else 0)
train['срочн'] = train.description.apply(lambda x: 1 if 'срочн' in x else 0)
train['полн'] = train.description.apply(lambda x: 1 if 'полн' in x else 0)
train['скидк'] = train.description.apply(lambda x: 1 if 'скидк' in x else 0)
train['диск'] = train.description.apply(lambda x: 1 if 'диск' in x else 0)
train['enginePower_1'] = train.enginePower.apply(lambda x: int(x[:-3]))**0.01

In [ ]:
test['гараж'] = test.description.apply(lambda x: 1 if 'гараж' in x else 0)
test['подарок'] = test.description.apply(lambda x: 1 if 'подарок' in x else 0)
test['торг'] = test.description.apply(lambda x: 1 if 'торг' in x else 0)
test['шины'] = test.description.apply(lambda x: 1 if ('шин' in x) or ('резин' in x) else 0)
test['дилер'] = test.description.apply(lambda x: 1 if 'дилер' in x else 0)
test['подогрев'] = test.description.apply(lambda x: 1 if 'подогрев' in x else 0)
test['обмен'] = test.description.apply(lambda x: 1 if 'обмен' in x else 0)
test['обслуж'] = test.description.apply(lambda x: 1 if 'обслуж' in x else 0)
test['срочн'] = test.description.apply(lambda x: 1 if 'срочн' in x else 0)
test['полн'] = test.description.apply(lambda x: 1 if 'полн' in x else 0)
test['скидк'] = test.description.apply(lambda x: 1 if 'скидк' in x else 0)
test['диск'] = test.description.apply(lambda x: 1 if 'диск' in x else 0)
test['enginePower_1'] = test.enginePower.apply(lambda x: int(x[:-3]))**0.01


In [ ]:
test.drop(['description'], axis=1, inplace=True)

In [ ]:
X = train.drop(['Price', 'description'], axis=1)
y = train.Price.values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=VAL_SIZE, shuffle=True, random_state=RANDOM_SEED)

In [ ]:
cat_features_ids = np.where(X_train.apply(pd.Series.nunique) < 3000)[0].tolist()

In [ ]:
cat_features_ids.remove(list(X_train.columns).index('enginePower_1'))
cat_features_ids.remove(list(X_train.columns).index('inuse'))

In [ ]:
model = CatBoostRegressor(iterations = ITERATIONS,
                          learning_rate = LR,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                         )
model.fit(X_train, y_train,
         cat_features=cat_features_ids,
         eval_set=(X_test, y_test),
         verbose_eval=100,
         use_best_model=True,
         plot=True
         )

In [ ]:
predict_submission_1 = model.predict(X_test)
mape(np.exp(y_test), np.exp(predict_submission_1))

In [ ]:
model.fit(X, y,
         cat_features=cat_features_ids,
         eval_set=(X, y),
         verbose_eval=100,
         use_best_model=True,
         plot=True
         )

Стекинг не улушил mape

In [ ]:
# estimators=[('b_gbr', RandomForestRegressor(random_state=RANDOM_SEED
#                       , n_estimators=300
#                       , min_samples_split=2
#                       , min_samples_leaf=1
#                       , max_features='sqrt'
#                       , max_depth=None
#                       , bootstrap=False))
#             ,('xgb', CatBoostRegressor(iterations = 3000,
#                           learning_rate = LR,
#                           random_seed = RANDOM_SEED,
#                           eval_metric='MAPE',
#                           bagging_temperature=1,
#                           one_hot_max_size=2,
#                           random_strength=1))]

# st_ensemble = StackingRegressor(estimators=estimators, n_jobs=-1)

    
# st_ensemble.fit(X_train, y_train)

# predict_ensemble = st_ensemble.predict(X_test)
# mape(np.exp(y_test), np.exp(predict_ensemble))

In [ ]:
predict_submission = np.exp(model.predict(test))

# Submission

In [ ]:
sample_submission['price'] = predict_submission
sample_submission.to_csv(f'submission_v{VERSION}.csv', index=False)
sample_submission.head(10)

In [ ]:
Результат можно улучшить если спарсить больше данных.